In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn.model_selection import cross_validate, GridSearchCV
import matplotlib.pylab as plt
%matplotlib inline 

In [2]:
import string
import os
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from nltk.stem.porter import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lzhan41\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lzhan41\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lzhan41\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lzhan41\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Plan 0.
1. tokenization/stem
2. Remove common words, remove the top 50 common words in all the docs 
3. vector the documents to the word of count 

In [3]:
text = pd.read_csv('train.txt', sep="\t", header=None, encoding = "ISO-8859-1")
text = pd.DataFrame(text[0].str.split(' ',1).tolist(), columns = ['ID', 'Raw_text'])
print(text.dtypes)

text['ID'] = text['ID'].astype(str).astype('int64')
text.head()

ID          object
Raw_text    object
dtype: object


ID                                           Raw_text
0  20030006121     Passive radio frequency identification syst...
1  20030009666     Methods and apparatus for efficient computa...
2  20030014337     Systems, methods and computer program produ...
3  20030019942     System and method for electronically readab...
4  20030033242     System and method for automated process of ...

In [4]:
# first convert the text column into a list of dictionary, ID is the key, text is the value 
# document is a list 
document=text.to_dict('records')


stop = set(stopwords.words('english'))
noun = ['NN', 'NNS', 'NNP', 'NNPS']
adj = ['JJ', 'JJR', 'JJS']
verb = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
stemmer = PorterStemmer()
for cell in document: 
    
    lowers = cell['Raw_text'].lower()
    tokens = nltk.word_tokenize(lowers)
    no_punct = [t for t in tokens if t not in string.punctuation]
    new_tokens = [w for w in no_punct if w.isalpha()] 
    no_stop = [w for w in new_tokens if not w in stop]
    length = [w for w in no_stop if 15>len(w)>3]
    
    ## remove the top 50 words from length 
#     word_counts = Counter(length)
#     top_50_words_frequency = word_counts.most_common(50)
#     top_50_words_ls = [word[0] for word in top_50_words_frequency] 
#     removel_50_words = list(set(length) - set(top_50_words_ls)) 
    
    stem = [stemmer.stem(w) for w in length]
    count_words=len(length) 
    doc = ' '.join(stem)
    

    pos_tag = nltk.pos_tag(nltk.word_tokenize(cell['Raw_text']))
    adj_list = [word[0] for word in pos_tag if word[1] in adj]
    noun_list = [word[0] for word in pos_tag if word[1] in noun]
    verb_list = [word[0] for word in pos_tag if word[1] in verb]
    
    cell.update(clean_text_stem=stem, number_wrds = count_words, 
                document = doc,
                number_puncuation = len(tokens)-len(no_punct), 
                number_raw_wrds = len(tokens) , number_adj=len(adj_list),
                number_noun=len(noun_list), number_verb=len(verb_list))


In [5]:
df = pd.DataFrame.from_dict(document)
df.columns

Index(['ID', 'Raw_text', 'clean_text_stem', 'document', 'number_adj',
       'number_noun', 'number_puncuation', 'number_raw_wrds', 'number_verb',
       'number_wrds'],
      dtype='object')

In [7]:
df.to_csv('0424.csv')


<center> <h1>Use TF-IDF to get ALL 1g word frequency </h1> </center>. 

In [8]:
norm_corpus = list(df['document'])
# norm_corpus

from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True, ngram_range=(1,1))
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
df_1w = pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)
df_1w.columns
df_1w.shape

(882, 17498)

<center> <h1>Use TF-IDF to get top 20 2-word frequency </h1> </center>. 

In [9]:
tv_2 = TfidfVectorizer(min_df=0., max_df=1., use_idf=True, max_features = 20, ngram_range=(2,2))
tv_matrix_2 = tv_2.fit_transform(norm_corpus)
tv_matrix_2 = tv_matrix_2.toarray()

vocab_2 = tv_2.get_feature_names()
df_2w = pd.DataFrame(np.round(tv_matrix_2, 2), columns=vocab_2)
df_2w.head()
df_2w.shape

(882, 20)

<center> <h1>Use TF-IDF to get top 10 3-word frequency </h1> </center>

In [10]:
tv_3 = TfidfVectorizer(min_df=0., max_df=1., use_idf=True, max_features = 10, ngram_range=(3,3))
tv_matrix_3 = tv_3.fit_transform(norm_corpus)
tv_matrix_3 = tv_matrix_3.toarray()

vocab_3 = tv_3.get_feature_names()
df_3w = pd.DataFrame(np.round(tv_matrix_3, 2), columns=vocab_3)
df_3w.head()
df_3w.shape

(882, 10)

<center> <h1>Combine all ngram features together </h1> </center>

In [11]:
# join df_1w, df_2w, df_32 
df_ngram = df_1w.join(df_2w).join(df_3w) 
df_final = pd.concat([df, df_ngram], axis=1)
df_final.shape
# df_final.head()

(882, 17538)

In [12]:
label = pd.read_csv('label.txt', sep="\t", header=None)
label.columns = ["ID", "Type"]
# print(label.dtypes)

df_final_2 = pd.merge(df_final, label, on='ID', how='left') 

# df_final_2.to_csv('final.csv')


df_final_2.drop(['Raw_text','clean_text_stem','document'], inplace=True, axis=1)
df_final_2.to_csv('final.csv')
df_final_2.head()

ID  number_adj  number_noun  number_puncuation  number_raw_wrds  \
0  20030006121         150          520                173             1743   
1  20030009666        1068         3389               1427            12517   
2  20030014337        1032         3309                897            10634   
3  20030019942         395         1267                344             4075   
4  20030033242        1865         7399               2570            24363   

   number_verb  number_wrds  aaai  aaby  aai  ...   claim wherein said  \
0          214          779   0.0   0.0  0.0  ...                 0.00   
1         1609         4801   0.0   0.0  0.0  ...                 0.00   
2         1191         4996   0.0   0.0  0.0  ...                 0.24   
3          439         1917   0.0   0.0  0.0  ...                 0.00   
4         3107        10965   0.0   0.0  0.0  ...                 0.35   

   comput program product  embodi present invent  method claim compris  \
0                    0.00                   0.00                  0.00   
1                    0.00                   0.09                  0.00   
2                    0.97                   0.03                  0.00   
3                    0.00                   0.00                  0.76   
4                    0.00                   0.26                  0.11   

   method claim wherein  mobil commun devic  mobil commun facil  \
0                  0.00                 0.0                 0.0   
1                  1.00                 0.0                 0.0   
2                  0.00                 0.0                 0.0   
3                  0.65                 0.0                 0.0   
4                  0.89                 0.0                 0.0   

   point sale devic  system claim wherein  Type  
0               0.0                   0.0     8  
1               0.0                   0.0     0  
2               0.0                   0.0     8  
3               0.0                   0.0     8  
4               0.0                   0.0     6  

[5 rows x 17535 columns]

In [12]:
df_final_2.shape

(882, 17535)

### Downsample to 34 cases for each category 

In [13]:
df_final_2.Type.value_counts()

df_class_0 = df_final_2[df_final_2['Type'] ==0]
df_class_0_under = df_class_0.sample(34)  

df_class_8 = df_final_2[df_final_2['Type'] ==8]
df_class_8_under = df_class_8.sample(34) 


df_class_1 = df_final_2[df_final_2['Type'] ==1]
df_class_1_under = df_class_1.sample(34) 


df_class_2 = df_final_2[df_final_2['Type'] ==2]
df_class_2_under = df_class_2.sample(34) 
df_class_2_under

df_class_3 = df_final_2[df_final_2['Type'] ==3]
df_class_3_under = df_class_3.sample(34) 
df_class_3_under 


df_class_6 = df_final_2[df_final_2['Type'] ==6]
df_class_6_under = df_class_6.sample(34) 
df_class_6_under  


df_class_5 = df_final_2[df_final_2['Type'] ==5]
df_class_5_under = df_class_5.sample(34) 
df_class_5_under  

df_class_4 = df_final_2[df_final_2['Type'] ==4]
df_class_4_under = df_class_4.sample(34) 
# df_class_4_under  

df_class_7 = df_final_2[df_final_2['Type'] ==7]
df_class_7_under = df_class_7.sample(34) 


df_final_2_under = pd.concat([df_class_0_under,df_class_1_under, df_class_2_under,df_class_3_under,
                              df_class_4_under, df_class_5_under,df_class_6_under,df_class_7_under,
                             df_class_8_under], axis=0)

df_final_2_under.head()

df_final_2_under.Type.value_counts()

8    34
7    34
6    34
5    34
4    34
3    34
2    34
1    34
0    34
Name: Type, dtype: int64

In [ ]:
# df_final_2_under.to_csv('downsample.csv')
df_final_2_under.shape 
# df_final_2_under.Type = df_final_2_under['Type'].astype('category') 
# df_final_2_under.dtypes



In [14]:
from sklearn.model_selection import train_test_split

X=df_final_2_under[list(df_final_2_under.columns)[1:-1]]  # Features
y=df_final_2_under['Type']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 7) 

In [52]:
# from sklearn.linear_model import LogisticRegression 
# from sklearn import metrics 
# from sklearn.metrics import classification_report
# import statsmodels.api as sm 

# X_train = sm.add_constant(X_train)
# lm_1 = sm.OLS(y_train, X_train).fit()
# lm_1.summary()

In [15]:
from sklearn.ensemble import RandomForestClassifier
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100,random_state=0)

# #Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.6290322580645161


### Feature selection through RF model 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel  
sel = SelectFromModel(RandomForestClassifier(n_estimators=100,random_state = 0)) 
sel.fit(X_train,y_train) 
sel.get_support() # return true is one feature is important than average feature importance
selected_feat= X_train.columns[(sel.get_support())]
# list(selected_feat) 
# print(len(selected_feat))

In [387]:
pd.set_option('display.max_rows', None)
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_ls = list(feature_imp[:300].index)
feature_ls.append('Type') 
df_select = df_final_2_under[feature_ls]

In [306]:
from sklearn.model_selection import train_test_split

X=df_final_2_under[list(selected_feat)]  # Features
y=df_final_2_under['Type']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 


from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100,random_state=0)

# #Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7258064516129032
              precision    recall  f1-score   support

           0       0.44      0.67      0.53         6
           1       0.40      0.33      0.36         6
           2       0.75      1.00      0.86         6
           3       1.00      0.57      0.73         7
           4       1.00      0.91      0.95        11
           5       0.90      0.90      0.90        10
           6       0.73      1.00      0.84         8
           7       0.67      0.67      0.67         3
           8       0.00      0.00      0.00         5

   micro avg       0.73      0.73      0.73        62
   macro avg       0.65      0.67      0.65        62
weighted avg       0.72      0.73      0.71        62



## Try Gradient boost Method  

In [261]:
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn.model_selection import cross_validate, cross_val_score   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   
#Perforing grid search
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import KFold 

In [581]:
from sklearn.model_selection import GridSearchCV
X=df_final_2_under[list(selected_feat)]  # Features
y=df_final_2_under['Type']   # Labels

# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [80, 90, 100],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100,500,1000],
    'learning_rate': [0.25, 0.5, 0.75],
    'subsample':[0.7,0.8,0.9]
}
# Create a based model
gb = GradientBoostingClassifier(random_state = 0)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = gb, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(X, y)

Fitting 3 folds for each of 1458 candidates, totalling 4374 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   40.9s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 1961 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done 3257 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 23.2min
[Parallel(n_jobs=-1)]: Done 4374 out of 4374 | elapsed: 26.6min finished
C:\Users\lzhan41\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [80, 90, 100], 'max_features': [2, 3], 'min_samples_leaf': [3, 4, 5], 'min_samples_split': [8, 10, 12], 'n_estimators': [100, 500, 1000], 'learning_rate': [0.25, 0.5, 0.75], 'subsample': [0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [281]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
cv_method = KFold(n_splits=3, shuffle=True)
model = grid_search.best_estimator_ 

model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("Classification Report")
print(classification_report(y_test, y_pred))
print(grid_search.best_score_)
# print(accuracy_score(y_test, y_pred))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.2s finished
C:\Users\lzhan41\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Classification Report
              precision    recall  f1-score   support

           0       0.45      1.00      0.62         5
           1       0.75      0.20      0.32        15
           2       0.50      0.83      0.62         6
           3       0.89      0.73      0.80        11
           4       0.78      1.00      0.88         7
           5       0.78      1.00      0.88         7
           6       1.00      0.75      0.86        12
           7       0.83      1.00      0.91         5
           8       0.50      0.56      0.53         9

   micro avg       0.70      0.70      0.70        77
   macro avg       0.72      0.79      0.71        77
weighted avg       0.75      0.70      0.68        77

0.6764705882352942


In [706]:
grid_search.best_params_

{'learning_rate': 0.25,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 4,
 'min_samples_split': 8,
 'n_estimators': 100,
 'subsample': 0.9}

In [319]:
df_final_2_under['Type'].nunique()

9